In [1]:
 ## IMPORTANDO PACOTES
    
## importantando function para poder realizar funções 
from pyspark.sql.functions import *
    
from pyspark.sql.types import *

## importando HIVEContext   que adiciona suporte para encontrar tabelas no MetaStore e escrever consultas usando o HiveQL.

from pyspark.sql import HiveContext

import pandas as pd  # .................................    importando biblioteca pandas 

sqlContext = HiveContext(sc)




Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1594167084901_0015,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.
/usr/bin/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')

### CRIANDO TABELA TOTAL DE SOLICITAÇÕES EM GERAL POR OPERADORA, ANUAL

In [16]:
%%sql


CREATE TABLE total_solic_uf AS SELECT T.ano, R.uf, O.operadora,C.tipo, SUM(quantidade) as total from hvfct_solicitacao F \
    JOIN hvdim_tempo T ON F.SK_TEMPO = T.SK_TEMPO \
    JOIN hvdim_regiao R ON F.SK_UF = R.SK_UF \
    JOIN hvdim_operadora O  ON F.SK_OPERADORA = O.SK_OPERADORA \
    JOIN hvdim_chamado C ON F.SK_CHAMADO = C.SK_CHAMADO  \
    GROUP BY T.ano,R.uf,O.operadora, C.tipo ORDER BY  total desc

### Criando tabela com o total de 'Reclamação' por operadora anual

In [20]:
%%sql

CREATE TABLE total_reclamacao_servico_operadora as SELECT T.ano, O.operadora,C.servico, C.motivo,C.tipo, sum(quantidade) as total from hvfct_solicitacao F \
    JOIN hvdim_operadora O ON F.SK_OPERADORA = O.SK_OPERADORA \
    JOIN hvdim_tempo T ON F.SK_TEMPO = T.SK_TEMPO \
    JOIN hvdim_chamado C ON F.SK_CHAMADO = C.SK_CHAMADO \
    WHERE C.tipo == "Reclamação" GROUP BY T.ano,O.operadora, C.servico,C.motivo,C.tipo ORDER BY total desc

### Criando tabela com o total de reclamações por operadora do serviço (celular pós pago)

In [20]:
%%sql

CREATE TABLE reclamacao_total_pos_pago AS SELECT O.operadora,C.servico, C.tipo,T.ano,C.motivo,sum(quantidade) AS total FROM hvfct_solicitacao F \
JOIN  hvdim_operadora O ON F.SK_OPERADORA = O.SK_OPERADORA \
JOIN hvdim_chamado C ON F.SK_CHAMADO = C.SK_CHAMADO \
JOIN hvdim_tempo T ON F.SK_TEMPO = T.SK_TEMPO \
WHERE C.tipo == "Reclamação" AND C.servico == "Celular Pós-Pago"  GROUP BY O.operadora,C.servico, C.tipo, T.ano,C.motivo ORDER BY total

### Criando tabela com o total de reclamações por operadora do serviço (TV ASSINATURA)

In [21]:
%%sql

CREATE TABLE reclamacao_total_TVassinatura  AS  SELECT O.operadora,C.servico, C.tipo,T.ano,C.motivo,sum(quantidade) AS total FROM hvfct_solicitacao F \
JOIN  hvdim_operadora O ON F.SK_OPERADORA = O.SK_OPERADORA \
JOIN hvdim_chamado C ON F.SK_CHAMADO = C.SK_CHAMADO \
JOIN hvdim_tempo T ON F.SK_TEMPO = T.SK_TEMPO \
WHERE C.tipo == "Reclamação" AND C.servico == "TV por Assinatura"  GROUP BY O.operadora,C.servico, C.tipo, T.ano ,C.motivo ORDER BY total

### Criando tabela com o total de reclamações por operadora do serviço (TELEFONE FIXO)

In [23]:
%%sql

CREATE TABLE reclamacao_total_TELFIXO  AS  SELECT O.operadora,C.servico, C.tipo,T.ano,C.motivo, sum(quantidade) AS total FROM hvfct_solicitacao F \
JOIN  hvdim_operadora O ON F.SK_OPERADORA = O.SK_OPERADORA \
JOIN hvdim_chamado C ON F.SK_CHAMADO = C.SK_CHAMADO \
JOIN hvdim_tempo T ON F.SK_TEMPO = T.SK_TEMPO \
WHERE C.tipo == "Reclamação" AND C.servico == "Telefone Fixo"  GROUP BY O.operadora,C.servico, C.tipo, T.ano,C.motivo ORDER BY total

### Criando tabela total de reclamação anual e mensalmente

In [52]:
%%sql

CREATE TABLE recl_total_ano_mes_operadora SELECT T.ano, CASE T.mes WHEN 1 THEN 'Janeiro' WHEN 2 THEN 'Fevereiro' \
                                WHEN 3 THEN 'Marco'   WHEN 4 THEN 'Abril' \
                                WHEN 5 THEN 'Maio'    WHEN 6 THEN 'Junho' \
                                WHEN 7 THEN 'Julho'   WHEN 8 THEN 'Agosto' \
                                WHEN 9 THEN 'Setembro'   WHEN 10 THEN 'Outubro' \
                                WHEN 11 THEN 'Novembro'   WHEN 12 THEN 'Dezembro' \
                                END AS name_mes, O.operadora, SUM(quantidade) AS total FROM hvfct_solicitacao F \
                            JOIN hvdim_tempo T ON F.SK_TEMPO = T.SK_TEMPO \
                            JOIN hvdim_chamado C ON F.SK_CHAMADO = C.SK_CHAMADO \
                            JOIN hvdim_operadora O ON F.SK_OPERADORA = O.SK_OPERADORA \
                            WHERE C.tipo == "Reclamação" \
                            GROUP BY T.ano, name_mes, O.operadora ORDER BY total 
                                 

### Criando tabela com o total de reclamações por operadora do serviço (Banda Larga fixa)

In [24]:
%%sql

CREATE TABLE reclamacao_total_BandaLarga AS SELECT O.operadora,C.servico, C.tipo,T.ano, C.motivo,sum(quantidade) AS total FROM hvfct_solicitacao F \
JOIN  hvdim_operadora O ON F.SK_OPERADORA = O.SK_OPERADORA \
JOIN hvdim_chamado C ON F.SK_CHAMADO = C.SK_CHAMADO \
JOIN hvdim_tempo T ON F.SK_TEMPO = T.SK_TEMPO \
WHERE C.tipo == "Reclamação" AND C.servico == "Banda Larga Fixa"  GROUP BY O.operadora,C.servico, C.tipo, T.ano,C.motivo ORDER BY total

### Criando tabela com o total de reclamações por operadora do serviço (celular pré pago)


In [18]:
%%sql

CREATE TABLE reclamacao_total_pre_pago AS SELECT O.operadora,C.servico, C.tipo,T.ano,C.motivo, sum(quantidade) AS total FROM hvfct_solicitacao F \
JOIN  hvdim_operadora O ON F.SK_OPERADORA = O.SK_OPERADORA \
JOIN hvdim_chamado C ON F.SK_CHAMADO = C.SK_CHAMADO \
JOIN hvdim_tempo T ON F.SK_TEMPO = T.SK_TEMPO \
WHERE C.tipo == "Reclamação" AND C.servico == "Celular Pré-Pago"  GROUP BY O.operadora,C.servico, C.tipo, T.ano,C.motivo ORDER BY total